# Ch.6 Financial Time Series

In [ ]:
#Choose the defauly plotting style
from pylab import plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

## pandas Basics

In [ ]:
import numpy as np, pandas as pd
#import pandas as pd

### First Steps with DataFrame Class

In [ ]:
df = pd.DataFrame([10, 20, 30, 40], columns=['numbers'],
                  index=['a', 'b', 'c', 'd'])
df

In [ ]:
df.index  # the index values

In [ ]:
df.columns  # the column names

In [ ]:
df.loc['c']  # selection via index, in Python2.7, this command line is df.ix['c']

In [ ]:
df.loc[['a', 'd']]  # selection of multiple indices

In [ ]:
df.loc[df.index[1:3]]  # selection via Index object

In [ ]:
df.sum()  # sum per column

In [ ]:
df.apply(lambda x: x ** 2)  # square of every element

In [ ]:
df ** 2  # again square, this time NumPy-like

In [ ]:
df['floats'] = (1.5, 2.5, 3.5, 4.5)
  # new column is generated, and share the same index
df

In [ ]:
df['floats']  # selection of column, and index will come with it

In [ ]:
df['names'] = pd.DataFrame(['Yves', 'Guido', 'Felix', 'Francesc'],
                           index=['d', 'a', 'b', 'c'])
# if the index is duplicated, it will report 'error'
df

In [1]:
df.append({'numbers': 100, 'floats': 5.75, 'names': 'Henry'},
               ignore_index=True)
  # temporary object; df not changed, and the index is initalzied as by their order

NameError: name 'df' is not defined

In [ ]:
df = df.append(pd.DataFrame({'numbers': 100, 'floats': 5.75,
                             'names': 'Henry'}, index=['z',]))
#here it is not temporary, because the .append() result is assigned to df
df

In [ ]:
df.join(pd.DataFrame([1, 4, 9, 16, 25],
            index=['a', 'b', 'c', 'd', 'y'],
            columns=['squares',]))
  # temporary object

In [ ]:
df = df.join(pd.DataFrame([1, 4, 9, 16, 25],
                    index=['a', 'b', 'c', 'd', 'y'],
                    columns=['squares',]),
                    how='outer')
df

In [ ]:
df[['numbers', 'squares']].mean()
  # column-wise mean

In [ ]:
df[['numbers', 'squares']].std()
  # column-wise standard deviation

### Second Steps with DataFrame Class

In [ ]:
a = np.random.standard_normal((9, 4))
a.round(6)

In [ ]:
df = pd.DataFrame(a)
df

In [ ]:
df.columns = ['No1', 'No2', 'No3', 'No4']
df

In [ ]:
df['No2'].iloc[3]  # value in column No2 at index position 3

In [ ]:
dates = pd.date_range('2015-1-1', periods=9, freq='M')
dates

In [ ]:
df.index = dates
df

In [ ]:
np.array(df).round(6)

### Basic Analytics

In [ ]:
df.sum()

In [ ]:
df.mean()

In [ ]:
df.cumsum()

In [ ]:
df.describe()

In [ ]:
np.sqrt(abs(df))

In [ ]:
np.sqrt(abs(df)).sum()

In [ ]:
%matplotlib inline
df.cumsum().plot(lw=2.0, grid=True)
# tag: dataframe_plot
# title: Line plot of a DataFrame object

### Series Class

In [ ]:
type(df)

In [ ]:
df['No1']

In [ ]:
type(df['No1'])

In [ ]:
import matplotlib.pyplot as plt
df['No1'].cumsum().plot(style='r', lw=2., grid=True)
plt.xlabel('date')
plt.ylabel('value')
# tag: time_series
# title: Line plot of a Series object

### GroupBy Operations

In [ ]:
df['Quarter'] = ['Q1', 'Q1', 'Q1', 'Q2', 'Q2', 'Q2', 'Q3', 'Q3', 'Q3']
df

In [ ]:
groups = df.groupby('Quarter')

In [ ]:
groups.mean()

In [ ]:
groups.max()

In [ ]:
groups.size()

In [ ]:
df['Odd_Even'] = ['Odd', 'Even', 'Odd', 'Even', 'Odd', 'Even',
                  'Odd', 'Even', 'Odd']

In [ ]:
groups = df.groupby(['Quarter', 'Odd_Even'])

In [ ]:
groups.size()

In [ ]:
groups.mean()

## Financial Data

In [ ]:
# data from Thomson Reuters Eikon API
raw = pd.read_csv('source/tr_eikon_eod_data.csv',
                 index_col=0, parse_dates=True)
raw.info()

In [ ]:
data = pd.DataFrame(raw['.SPX'])
data.columns = ['Close']

In [ ]:
data.tail()

In [ ]:
data['Close'].plot(figsize=(8, 5), grid=True);
# tag: dax
# title: Historical DAX index levels

In [ ]:
%time data['Return'] = np.log(data['Close'] / data['Close'].shift(1))

In [ ]:
data[['Close', 'Return', 'Return']].tail()

In [ ]:
data[['Close', 'Return']].plot(subplots=True, style='b',
                              figsize=(8, 5), grid=True);
# tag: dax_returns
# title: The S&P 500 index and daily log returns

In [ ]:
data['42d'] = data['Close'].rolling(window=42).mean()
data['252d'] = data['Close'].rolling(window=252).mean()

In [ ]:
data[['Close', '42d', '252d']].tail()

In [ ]:
data[['Close', '42d', '252d']].plot(figsize=(8, 5), grid=True)
# tag: dax_trends
# title: The S&P index and moving averages

In [ ]:
import math
data['Mov_Vol'] = data['Return'].rolling(window=252).std() * math.sqrt(252)
  # moving annual volatility

In [ ]:
data[['Close', 'Mov_Vol', 'Return']].plot(subplots=True, style='b',
                                         figsize=(8, 7), grid=True);
# tag: dax_mov_std
# title: The S&P index and moving, annualized volatility

## Regression Analysis

In [ ]:
import pandas as pd

In [ ]:
# data from Thomson Reuters Eikon API
raw = pd.read_csv('source/tr_eikon_eod_data.csv',
                 index_col=0, parse_dates=True)
spx = pd.DataFrame(raw['.SPX'])

In [ ]:
np.round(spx.tail())

In [ ]:
vix = pd.DataFrame(raw['.VIX'])
vix.info()

In [ ]:
data = spx.join(vix)

In [ ]:
data.tail()

In [ ]:
data.plot(subplots=True, grid=True, style='b', figsize=(8, 6));
# tag: spx_vix
# title: The S&P 500 Index and the VIX volatility index

In [ ]:
rets = np.log(data / data.shift(1)) 
rets.head()

In [ ]:
rets.dropna(inplace=True)

In [ ]:
rets.plot(subplots=True, grid=True, style='b', figsize=(8, 6));
# tag: es50_vs_rets
# title: Log returns of S&P500 and VIX

In [ ]:
import numpy as np

In [ ]:
xdat = rets['.SPX'].values
ydat = rets['.VIX'].values
reg = np.polyfit(x=xdat, y=ydat, deg=1)
reg

In [ ]:
plt.plot(xdat, ydat, 'r.')
ax = plt.axis()  # grab axis values
x = np.linspace(ax[0], ax[1] + 0.01)
plt.plot(x, np.polyval(reg, x), 'b', lw=2)
plt.grid(True)
plt.axis('tight')
plt.xlabel('S&P 500 returns')
plt.ylabel('VIX returns')
# tag: scatter_rets
# title: Scatter plot of log returns and regression line

In [ ]:
rets.corr()

In [ ]:
rets['.SPX'].rolling(window=252).corr(rets['.VIX']).plot(grid=True, style='b')
# tag: roll_corr
# title: Rolling correlation between S&P 500 and VIX

## High Frequency Data

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
%matplotlib inline

In [ ]:
# data from FXCM Forex Capital Markets Ltd.
eur_usd = pd.read_csv('source/fxcm_eur_usd_tick_data.csv',
                     index_col=0, parse_dates=True)

In [ ]:
eur_usd.info()

In [ ]:
eur_usd['Mid'] = eur_usd.mean(axis=1)

In [ ]:
eur_usd['Mid'].plot(figsize=(10, 6));
# tag: eur_uusd
# title: EUR/USD tick data for two hours

In [ ]:
eur_usd_resam = eur_usd.resample(rule='1min', label='last').last()
eur_usd_resam.head()

In [ ]:
eur_usd_resam['Mid'].plot(grid=True)
# tag: eur_usd_resam
# title: Resampled EUR/USD exchange rate tick data

In [ ]:
def reversal(x):
    return 2 * 1.16 - x

In [ ]:
eur_usd_resam['Mid'].apply(reversal).plot()
# tag: aapl_resam_apply
# title: Resampled EUR/USD exchange rate tick data with function applied to it

## Conclusions

## Further Reading

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

**Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>

**Listed Volatility and Variance Derivatives** |
<a href="http://lvvd.tpq.io" target="_blank">Listed VV Derivatives @ Wiley Finance</a>

**Python Training** |
<a href="http://training.tpq.io" target="_blank">Python for Finance University Certificate</a>